Vamos a realizar peticiones y transformaciones sobre datos de la siguiente api:

https://newcastle.urbanobservatory.ac.uk/api_docs/

## Ejercicio 1

Familiarizarse con la api del Urban Observatory y realizar una petición sencilla

## Ejercicio 2

Objetivo: Obtener datos de sensores meteorológicos y realizar un análisis básico.

Utiliza el endpoint específico para obtener datos de temperatura, humedad, o viento.
Almacena los datos en un DataFrame de Pandas.
Calcula medidas estadísticas básicas (media, mediana, máximo, mínimo).


## Ejercicio 3

Obtén datos de tráfico de un sensor específico.
Almacena los datos en un DataFrame y convierte las marcas de tiempo al formato correcto.
Grafica la cantidad de vehículos detectados por hora a lo largo del día

## Ejercicio 4

Utiliza una biblioteca de Python para visualización de mapas como Folium.
Representa la ubicación de los sensores en un mapa y colorea o dimensiona los marcadores según la intensidad de un factor específico (por ejemplo, la contaminación).
Crea un mapa interactivo que permita a los usuarios explorar los diferentes datos